In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [5]:
enrol_data<-read_csv("Data/Enrolment_Data.csv")
head(enrol_data)

Rows: 13 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (2): year, enrolment

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


year,enrolment
<dbl>,<dbl>
2012,8438
2013,9371
2014,10903
2015,12117
2016,13182
2017,14685


In [6]:
glimpse(enrol_data)
summary(enrol_data)

Rows: 13
Columns: 2
$ year      <dbl> 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, …
$ enrolment <dbl> 8438, 9371, 10903, 12117, 13182, 14685, 15405, 16098, 15504,…


      year        enrolment    
 Min.   :2012   Min.   : 8438  
 1st Qu.:2015   1st Qu.:12117  
 Median :2018   Median :15405  
 Mean   :2018   Mean   :14086  
 3rd Qu.:2021   3rd Qu.:16357  
 Max.   :2024   Max.   :17243  